## CAIXABANK Hackathon - Data Science

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
project_path = "/content/drive/MyDrive/Data_Science_&_AI/NUWE/"
train_data_dir_csv = [project_path + 'Data/CSV/' + x for x in os.listdir(project_path+'Data/CSV') if 'train' in x]
test_data_dir_csv = [project_path + 'Data/CSV/' + x for x in os.listdir(project_path+'Data/CSV') if 'test' in x][0]
train_data_dir_pdf = [project_path + 'Data/pdf/' + x for  x in os.listdir(project_path+'Data/pdf')]
train_data_api = project_path + 'Data/json/links.txt'

### IMPORT MODULES

In [1]:
import utils 

In [2]:
utils.train.a()

jola


### DATA LOAD

#### STOCK

#### TWEETS

#### COMBINE DATA

### EDA

## MODELS

#### RESULTS

## NEXT STEPS